# Solution — Task 01: pytest Basics

## Setup

In [ ]:
import subprocess, sys, os, textwrap, tempfile, pathlib

FIXTURES = os.path.abspath(os.path.join("..", "fixtures", "input"))
if not os.path.exists(FIXTURES):
    FIXTURES = os.path.abspath(os.path.join("fixtures", "input"))
sys.path.insert(0, FIXTURES)

from sample_module import clean_text, tokenize, count_words, extract_emails, truncate, is_palindrome, mask_pii

def run_pytest(test_code: str, extra_args: list[str] | None = None):
    with tempfile.TemporaryDirectory() as td:
        p = pathlib.Path(td) / "test_tmp.py"
        p.write_text(textwrap.dedent(test_code))
        cmd = [sys.executable, "-m", "pytest", str(p), "-v", "--tb=short", "--no-header"]
        if extra_args:
            cmd.extend(extra_args)
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=30)
        print(result.stdout + result.stderr)
        return result.returncode

print("Setup complete. Fixtures at:", FIXTURES)

## Solution 1.1: Test `clean_text`

In [ ]:
test_clean_text = f'''
import sys
sys.path.insert(0, "{FIXTURES}")
from sample_module import clean_text

def test_extra_spaces():
    assert clean_text("  hello   world  ") == "hello world"

def test_empty_string():
    assert clean_text("") == ""

def test_tabs_and_newlines():
    assert clean_text("hello\\tworld\\nfoo") == "hello world foo"

def test_single_word():
    assert clean_text("  word  ") == "word"
'''

rc = run_pytest(test_clean_text)
assert rc == 0, "Tests should pass"
print("Task 1.1 passed!")

## Solution 1.2: Test `tokenize` with `parametrize`

In [ ]:
test_tokenize = f'''
import sys, pytest
sys.path.insert(0, "{FIXTURES}")
from sample_module import tokenize

@pytest.mark.parametrize("text, expected", [
    ("hello world", ["hello", "world"]),
    ("", []),
    ("word", ["word"]),
    ("Hello World", ["hello", "world"]),
    ("  extra   spaces  ", ["extra", "spaces"]),
    ("ALL CAPS", ["all", "caps"]),
])
def test_tokenize(text, expected):
    assert tokenize(text) == expected

def test_tokenize_no_lowercase():
    assert tokenize("Hello World", lowercase=False) == ["Hello", "World"]
'''

rc = run_pytest(test_tokenize)
assert rc == 0, "Tests should pass"
print("Task 1.2 passed!")

## Solution 1.3: Test Exceptions

In [ ]:
test_exceptions = f'''
import sys, pytest
sys.path.insert(0, "{FIXTURES}")
from sample_module import clean_text, tokenize, truncate

def test_clean_text_type_error():
    with pytest.raises(TypeError):
        clean_text(123)

def test_tokenize_type_error():
    with pytest.raises(TypeError):
        tokenize(None)

def test_truncate_negative_length():
    with pytest.raises(ValueError, match="non-negative"):
        truncate("hello", max_length=-1)
'''

rc = run_pytest(test_exceptions)
assert rc == 0, "Tests should pass"
print("Task 1.3 passed!")

## Solution 1.4: Test `extract_emails` and `mask_pii`

In [ ]:
test_pii = f'''
import sys
sys.path.insert(0, "{FIXTURES}")
from sample_module import extract_emails, mask_pii

def test_extract_no_emails():
    assert extract_emails("no email here") == []

def test_extract_one_email():
    assert extract_emails("contact user@example.com please") == ["user@example.com"]

def test_extract_multiple_emails():
    text = "reach a@b.com or c@d.org"
    assert extract_emails(text) == ["a@b.com", "c@d.org"]

def test_mask_pii_email():
    assert mask_pii("contact user@example.com") == "contact [EMAIL]"

def test_mask_pii_phone():
    assert mask_pii("call 123-456-7890") == "call [PHONE]"

def test_mask_pii_both():
    text = "email: a@b.com phone: 123.456.7890"
    result = mask_pii(text)
    assert "[EMAIL]" in result
    assert "[PHONE]" in result
    assert "a@b.com" not in result
'''

rc = run_pytest(test_pii)
assert rc == 0, "Tests should pass"
print("Task 1.4 passed!")

## Solution 1.5: Test `is_palindrome` and `truncate`

In [ ]:
test_misc = f'''
import sys
sys.path.insert(0, "{FIXTURES}")
from sample_module import is_palindrome, truncate

def test_palindrome_true():
    assert is_palindrome("racecar") is True

def test_palindrome_with_punctuation():
    assert is_palindrome("A man, a plan, a canal: Panama") is True

def test_palindrome_false():
    assert is_palindrome("hello") is False

def test_truncate_short():
    assert truncate("hi", max_length=10) == "hi"

def test_truncate_long():
    assert truncate("hello world", max_length=8) == "hello..."

def test_truncate_exact():
    assert truncate("hello", max_length=5) == "hello"
'''

rc = run_pytest(test_misc)
assert rc == 0, "Tests should pass"
print("Task 1.5 passed!")